In [1]:
import tensorflow as tf
#import tensorflow_hub as hub
import requests
print(tf.__version__)
import zipfile
import requests
import os
import time
import pandas as pd
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
import numpy as np
from PIL import Image
import pickle
from tensorflow.keras.models import load_model, Model
from PIL import Image
from PIL.PngImagePlugin import PngImageFile
import matplotlib.pyplot as plt
import glob
from functools import partial
import nltk

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
random_seed=4321
fix_random_seed(random_seed)

2.2.1


In [2]:
# Downloading the data
# http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz

import os
import requests
import tarfile

import shutil

# Retrieve the data
if not os.path.exists(os.path.join('data', 'lm','CBTest.tgz')):
    url = "http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists(os.path.join('data','lm')):
        os.mkdir(os.path.join('data','lm'))
    
    # Write to a file
    with open(os.path.join('data', 'lm', 'CBTest.tgz'), 'wb') as f:
        f.write(r.content)
          
else:
    print("The tar file already exists.")
    
if not os.path.exists(os.path.join('data', 'lm', 'CBTest')):
    # Write to a file
    tarf = tarfile.open(os.path.join("data","lm","CBTest.tgz"))
    tarf.extractall(os.path.join("data","lm"))  
else:
    print("The extracted data already exists")


The tar file already exists.
The extracted data already exists


In [3]:
def read_data(path):
    stories = []

    with open(path, 'r') as f:    
        s = []
        start_capture = False    
        for row in f:

            if start_capture:
                s.append(row)

            if row.startswith("_BOOK_TITLE_"):
                if len(s)>0:
                    stories.append(' '.join(s).lower())            
                s = []
                start_capture=True            

    if len(s)>0:
        stories.append(' '.join(s).lower())  
    
    return stories

stories = read_data(os.path.join('data','lm','CBTest','data','cbt_train.txt'))
val_stories = read_data(os.path.join('data','lm','CBTest','data','cbt_valid.txt'))
test_stories = read_data(os.path.join('data','lm','CBTest','data','cbt_test.txt'))

In [4]:
print("Collected {} stories (train)".format(len(stories)))
print("Collected {} stories (valid)".format(len(val_stories)))
print("Collected {} stories (test)".format(len(test_stories)))
print(stories[0][:100])
print('\n', stories[10][:100])


Collected 98 stories (train)
Collected 5 stories (valid)
Collected 5 stories (test)
chapter i. -lcb- chapter heading picture : p1.jpg -rcb- how the fairies were not invited to court .


 a tale of the tontlawald long , long ago there stood in the midst of a country covered with lakes a 


In [5]:
from itertools import chain
from collections import Counter

def ngrams(text, n):
    return [text[i:i+n] for i in range(0,len(text),n)]

test_string = "I like chocolates"
print("Original: {}".format(test_string))
for i in list(range(3)):
    print("\t{}-grams: {}".format(i+1, ngrams(test_string, i+1)))
    
text = chain(*[ngrams(s, 2) for s in stories])
cnt = Counter(text)
freq_df = pd.Series(list(cnt.values()), index=list(cnt.keys())).sort_values(ascending=False)
print(freq_df.head(n=10))
print("\nMedian: {}\n".format(freq_df.median()))
print(freq_df.describe(percentiles=[0.25,0.5,0.75,0.9]))

Original: I like chocolates
	1-grams: ['I', ' ', 'l', 'i', 'k', 'e', ' ', 'c', 'h', 'o', 'c', 'o', 'l', 'a', 't', 'e', 's']
	2-grams: ['I ', 'li', 'ke', ' c', 'ho', 'co', 'la', 'te', 's']
	3-grams: ['I l', 'ike', ' ch', 'oco', 'lat', 'es']
e     455292
 t    344971
he    310539
d     308390
th    284425
 a    268496
t     257788
s     227961
 h    192544
 s    182830
dtype: int64

Median: 136.5

count      1074.000000
mean      12106.919926
std       36358.817692
min           1.000000
25%           5.000000
50%         136.500000
75%        6406.750000
90%       34184.600000
max      455292.000000
dtype: float64


In [6]:
n_vocab = (freq_df>=10).sum()
print("Size of vocabulary: {}".format(n_vocab))

Size of vocabulary: 735


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

n = 2 #ngram length
tokenizer = Tokenizer(num_words=n_vocab, oov_token='unk', lower=False)
train_ngram_stories = [ngrams(s,n) for s in stories]
tokenizer.fit_on_texts(train_ngram_stories)

train_data_seq = tokenizer.texts_to_sequences(train_ngram_stories)

val_ngram_stories = [ngrams(s,n) for s in val_stories]
val_data_seq = tokenizer.texts_to_sequences(val_ngram_stories)

test_ngram_stories = [ngrams(s,n) for s in test_stories]
test_data_seq = tokenizer.texts_to_sequences(test_ngram_stories)

In [8]:
def get_tf_pipeline(data_seq, n_seq, batch_size=64, stride=1, shuffle=True):
        
    text_ds = tf.data.Dataset.from_tensor_slices(tf.ragged.constant(data_seq)) # tf.ragged.constant(data_seq)
    
    if shuffle:
        text_ds = text_ds.shuffle(buffer_size=len(data_seq)//2)
        
    text_ds = text_ds.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(x).window(n_seq+1,shift=stride)    )
    text_ds = text_ds.flat_map(lambda window: window.batch(n_seq+1, drop_remainder=True))
    
    #text_ds = text_ds.unbatch()
    
    if shuffle:
        text_ds = text_ds.shuffle(buffer_size=10*batch_size)
    
    text_ds = text_ds.batch(batch_size)
    text_ds = tf.data.Dataset.zip(text_ds.map(lambda x: (x[:,:-1], x[:, 1:]))).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return text_ds
    #return x,y



In [12]:
ds = get_tf_pipeline(train_data_seq, 5, batch_size=6)

for a in ds.take(3):

    print(a)


(<tf.Tensor: shape=(6, 5), dtype=int32, numpy=
array([[ 8, 12,  3, 12,  2],
       [11,  2,  9,  2,  4],
       [ 4,  3, 11,  2,  9],
       [28,  3, 10,  4,  2],
       [12,  2, 10, 14, 16],
       [16, 16,  3, 11,  2]], dtype=int32)>, <tf.Tensor: shape=(6, 5), dtype=int32, numpy=
array([[12,  3, 12,  2,  5],
       [ 2,  9,  2,  4,  7],
       [ 3, 11,  2,  9,  2],
       [ 3, 10,  4,  2,  9],
       [ 2, 10, 14, 16, 16],
       [16,  3, 11,  2,  9]], dtype=int32)>)
(<tf.Tensor: shape=(6, 5), dtype=int32, numpy=
array([[29, 29,  2,  7,  5],
       [ 2,  4,  7,  3,  2],
       [ 8,  3,  2, 21,  2],
       [ 2, 20,  7,  5,  8],
       [14, 16, 16,  3, 11],
       [23,  4,  3, 11,  2]], dtype=int32)>, <tf.Tensor: shape=(6, 5), dtype=int32, numpy=
array([[29,  2,  7,  5, 11],
       [ 4,  7,  3,  2, 10],
       [ 3,  2, 21,  2, 26],
       [20,  7,  5,  8, 17],
       [16, 16,  3, 11,  2],
       [ 4,  3, 11,  2,  9]], dtype=int32)>)
(<tf.Tensor: shape=(6, 5), dtype=int32, numpy=
array([

https://arxiv.org/pdf/1708.02182.pdf

In [9]:
import tensorflow.keras.backend as K

# https://gist.github.com/Gregorgeous/dbad1ec22efc250c76354d949a13cec3
class PerplexityMetric(tf.keras.metrics.Mean):
    """
    USAGE NOTICE: this metric accepts only logits for now (i.e. expect the same behaviour as from tf.keras.losses.SparseCategoricalCrossentropy with the a provided argument "from_logits=True", 
		here the same loss is used with "from_logits=True" enforced so you need to provide it in such a format)
    METRIC DESCRIPTION:
    Popular metric for evaluating language modelling architectures.
    More info: http://cs224d.stanford.edu/lecture_notes/LectureNotes4.pdf.
    DISCLAIMER: Original function created by Kirill Mavreshko in https://github.com/kpot/keras-transformer/blob/b9d4e76c535c0c62cadc73e37416e4dc18b635ca/example/run_gpt.py#L106. 
    My "contribution": I converted Kirill method's logic (and added a padding masking to to it) into this new Tensorflow 2.0 way of doing things via a stateful "Metric" object. This required making the metric a fully-fledged object by subclassing the Metric class. 
    """
    def __init__(self, name='perplexity', **kwargs):
      super(PerplexityMetric, self).__init__(name=name, **kwargs)
      self.cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')

    def _calculate_perplexity(self, real, pred):
      # The next 4 lines zero-out the padding from loss calculations, 
      # this follows the logic from: https://www.tensorflow.org/beta/tutorials/text/transformer#loss_and_metrics 			
      #mask = tf.math.logical_not(tf.math.equal(real, 0))
      loss_ = self.cross_entropy(real, pred)
      #print(loss_.shape)
      #mask = tf.cast(mask, dtype=loss_.dtype)
      #loss_ *= mask
      # Calculating the perplexity steps: 
      step1 = K.mean(loss_, axis=-1)
      step2 = K.exp(step1)
      perplexity = K.mean(step2)
      #perplexity = K.exp(K.mean(loss_))
      
      return perplexity 

    def update_state(self, y_true, y_pred, sample_weight=None):            
      perplexity = self._calculate_perplexity(y_true, y_pred)
      # Remember self.perplexity is a tensor (tf.Variable), so using simply "self.perplexity = perplexity" will result in error because of mixing EagerTensor and Graph operations 
      super(PerplexityMetric, self).update_state(perplexity)

In [12]:
        
K.clear_session()
n_seq=100
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=n_vocab+1, output_dim=512, input_shape=(None,)),
    # Defining an LSTM layer
    tf.keras.layers.GRU(1024, return_state=False, return_sequences=True),
    #tf.keras.layers.GRU(512, return_state=False, return_sequences=True),
    # Defining a Dense layer
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.8)),
    #tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.TimeDistributed(tf.keras.layers.Dropout(0.8)),
    tf.keras.layers.Dense(n_vocab, name='final_out'),
    tf.keras.layers.Activation(activation='softmax')
])
# Compile the model
model.compile(
    loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy', PerplexityMetric()]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 512)         376832    
_________________________________________________________________
gru (GRU)                    (None, None, 1024)        4724736   
_________________________________________________________________
dense (Dense)                (None, None, 512)         524800    
_________________________________________________________________
final_out (Dense)            (None, None, 735)         377055    
_________________________________________________________________
activation (Activation)      (None, None, 735)         0         
Total params: 6,003,423
Trainable params: 6,003,423
Non-trainable params: 0
_________________________________________________________________


In [47]:
p = PerplexityMetric()
true = [[0, 1,2],[0, 1,2]]
pred = [[[0.9, 0.1, 0.0], [0.3, 0.7, 0.0], [0.0, 0.1, 0.9]],[[0.9, 0.1, 0.0], [0.3, 0.7, 0.0], [0.0, 0.1, 0.9]]]
p.update_state(true, pred)
print(p.result())

tf.Tensor(1.2082006, shape=(), dtype=float32)


# GRU got to around 35% accuracy and 39 validation perplexity

5873 steps 128 batch size 50 stories

In [79]:
train_ds = get_tf_pipeline(train_data_seq[:50], n_seq, stride=25, batch_size=128)
valid_ds = get_tf_pipeline(val_data_seq, n_seq, stride=n_seq, batch_size=128)

os.makedirs('eval', exist_ok=True)

# Logging the performance metrics to a CSV file
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join('eval','1_language_modelling.log'))

monitor_metric = 'val_perplexity'
mode = 'min' 
print("Using metric={} and mode={} for EarlyStopping".format(monitor_metric, mode))

# Reduce LR callback
# This function keeps the initial learning rate for the first ten epochs
# and decreases it exponentially after that.
def scheduler(epoch, lr):  
    if epoch==0:
        return lr
    else:
        return lr * 0.1

#lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)


lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor=monitor_metric, factor=0.1, patience=2, mode=mode, min_lr=1e-8
)

# EarlyStopping itself increases the memory requirement
# restore_best_weights will increase the memory req for large models
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor=monitor_metric, patience=5, mode=mode, restore_best_weights=False
)

t1 = time.time()

model.fit(train_ds, epochs=50, 
          validation_data = valid_ds,
          callbacks=[es_callback, lr_callback, csv_logger])
t2 = time.time()

print("It took {} seconds to complete the training".format(t2-t1))

Using metric=val_perplexity and mode=min for EarlyStopping
Epoch 1/50
2936/2936 [==============================] - 341s 116ms/step - loss: 2.3857 - accuracy: 0.4313 - perplexity: 15.2877 - val_loss: 2.4790 - val_accuracy: 0.4113 - val_perplexity: 12.6962 - lr: 0.0010
Epoch 2/50
2936/2936 [==============================] - 343s 117ms/step - loss: 2.0692 - accuracy: 0.4853 - perplexity: 8.4284 - val_loss: 2.4818 - val_accuracy: 0.4150 - val_perplexity: 12.8089 - lr: 0.0010
Epoch 3/50
2936/2936 [==============================] - 344s 117ms/step - loss: 2.0456 - accuracy: 0.4889 - perplexity: 8.2086 - val_loss: 2.4728 - val_accuracy: 0.4130 - val_perplexity: 12.6940 - lr: 0.0010
Epoch 4/50
2936/2936 [==============================] - 345s 117ms/step - loss: 2.0390 - accuracy: 0.4891 - perplexity: 8.1284 - val_loss: 2.4252 - val_accuracy: 0.4215 - val_perplexity: 12.0610 - lr: 0.0010
Epoch 5/50
2936/2936 [==============================] - 347s 118ms/step - loss: 2.0380 - accuracy: 0.4885 - 

In [80]:
os.makedirs('models', exist_ok=True)
tf.keras.models.save_model(model, os.path.join('models', '2_gram_lm.h5'))

## Load model

In [10]:
model = tf.keras.models.load_model(os.path.join('models', '2_gram_lm.h5'), compile=False)
model.compile(
    loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy', PerplexityMetric()]
)

## Defining the inference model

In [11]:
inp = tf.keras.layers.Input(shape=(None,))
inp_state = tf.keras.layers.Input(shape=(1024,))

emb_layer = tf.keras.layers.Embedding(input_dim=n_vocab+1, output_dim=512, input_shape=(None,))
emb_out = emb_layer(inp)
    # Defining an LSTM layer
gru_layer = tf.keras.layers.GRU(1024, return_state=True, return_sequences=True)
gru_out, gru_state = gru_layer(emb_out, initial_state=inp_state)

dense_layer = tf.keras.layers.Dense(512, activation='relu')
dense_out = dense_layer(gru_out)
final_layer = tf.keras.layers.Dense(n_vocab, name='final_out')
final_out = final_layer(dense_out)
softmax_out = tf.keras.layers.Activation(activation='softmax')(final_out)

infer_model = tf.keras.models.Model(inputs=[inp, inp_state], outputs=[softmax_out, gru_state])

emb_layer.set_weights(model.get_layer('embedding').get_weights())
gru_layer.set_weights(model.get_layer('gru').get_weights())
dense_layer.set_weights(model.get_layer('dense').get_weights())
final_layer.set_weights(model.get_layer('final_out').get_weights())
infer_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 512)    376832      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1024)]       0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(None, None, 1024), 4724736     embedding[0][0]                  
                                                                 input_2[0][0]                

## Greedy decoding

In [177]:
text = ngrams(
    "CHAPTER I. Down the Rabbit-Hole Alice was beginning to get very tired of sitting by her sister on the bank  , and of having nothing to do : once or twice she had peeped into the book her sister was reading , but it had no pictures or conversations in it , ` and what is the use of a book , ' thought Alice ` without pictures or conversation ? ' ".lower(), 
    n
)

# CHAPTER I. Down the Rabbit-Hole Alice was beginning to get very tired of sitting by her sister on the bank
seq = tokenizer.texts_to_sequences([text])

# build up model state using the given string
print("Making {} predictions from input".format(len(seq[0])))

model.reset_states()
state = np.zeros(shape=(1,1024))
for c in seq[0]:    
    out, state = infer_model.predict([np.array([[c]]), state])

# get final prediction after feeding the input string
wid = int(np.argmax(out[0],axis=-1).ravel())
word = tokenizer.index_word[wid]
text.append(word)

x = np.array([[wid]])

for _ in range(500):

    out, state = infer_model.predict([x, state])
    # Get the word id and the word
    out_argsort = np.argsort(out[0], axis=-1).ravel()    
    #i = np.random.choice(list(range(-2,0)), p=out_argsort[-2:]/out_argsort[-2:].sum())    
    wid = int(out_argsort[-1])    
    word = tokenizer.index_word[wid]
    
    if word.endswith(' '):
        if np.random.normal()>0.5:
            width = 3
            i = np.random.choice(list(range(-width,0)), p=out_argsort[-width:]/out_argsort[-width:].sum())    
            wid = int(out_argsort[i])    
            word = tokenizer.index_word[wid]
    # Append the prediction
    text.append(word)
    
    # recursively make the current prediction the next input
    x = np.array([[wid]])
    
    
print(''.join(text))

Making 173 predictions from input
chapter i. down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank  , and of having nothing to do : once or twice she had peeped into the book her sister was reading , but it had no pictures or conversations in it , ` and what is the use of a book , ' thought alice ` without pictures or conversation ? ' unkunk. h. m. h. that 's all . ''
 `` i do n't know what it is , '' said the story girl , `` i 'm going to see them , and i 'l n't wait till i 'm going to take yourself on the water .
 i 'm going to see you again , and i 'll tell you what it is . ''
 ` whatsoeve , it 's all right , '' said mrs. jo , standing up and down they wended .
 `` therefore-rings them , '' said mrs. jo , smiling at the little girl .
 `` i 'm going to bed , '' said the king , `` and i 'm sure you 'll be able to discover what is the matter ... and i 'll tell younda williams , and i 'll be able to do it ... i 'm going to say that i have n't an

In [ ]:
https://gist.github.com/Gregorgeous/dbad1ec22efc250c76354d949a13cec3

## Beam search decoding

In [25]:
def beam_one_step(model, input_, state):    
    #print('a',input_,)
    #print('b',state)
    output, new_state = model.predict([input_, state])
    return output, new_state



def beam_search_v2(model, input_, state, beam_depth=5, beam_width=3, ignore_blank=True):
    
    def recursive_fn(input_, state, sequence, log_prob, i):
        
        if i == beam_depth:
            # Return the results            
            #print('end', sequence)
            results.append((list(sequence), state, np.exp(log_prob)))            
            return sequence, log_prob, state
        else:
            output, new_state = beam_one_step(model, input_, state)
            top_probs, top_ids = tf.nn.top_k(output, k=beam_width)
            top_probs, top_ids = top_probs.numpy().ravel(), top_ids.numpy().ravel()
            #print(top_probs, top_ids)
            for p, wid in zip(top_probs, top_ids):
                # we are going to assigne a very low probability whenever the same symbol is repeating                                
                new_log_prob = log_prob + np.log(p)
                if len(sequence)>0 and wid == sequence[-1]:
                    new_log_prob = new_log_prob + np.log(5e-1)
                    
                sequence.append(wid)                
                _ = recursive_fn(np.array([[wid]]), new_state, sequence, new_log_prob, i+1)                                         
                sequence.pop()
        
    
    results = []
    sequence = []
    log_prob = 0.0
    recursive_fn(input_, state, sequence, log_prob, 0)    

    results = sorted(results, key=lambda x: x[2], reverse=True)

    return results

In [33]:
text = ngrams(
    "CHAPTER I. Down the Rabbit-Hole Alice was beginning to get very tired of sitting by her sister on the bank ,".lower(), 
    #"CHAPTER I".lower(),
    n
)

# CHAPTER I. Down the Rabbit-Hole Alice was beginning to get very tired of sitting by her sister on the bank
seq = tokenizer.texts_to_sequences([text])

# build up model state using the given string
print("Making {} predictions from input".format(len(seq[0])))

#model.reset_states()
state = np.zeros(shape=(1,1024))
for c in seq[0]:    
    out, state = infer_model.predict([np.array([[c]]), state])

# get final prediction after feeding the input string
wid = int(np.argmax(out[0],axis=-1).ravel())
word = tokenizer.index_word[wid]
text.append(word)

x = np.array([[wid]])

for i in range(100):    
    result = beam_search_v2(infer_model, x, state, 7, 2)
    
    #best_beam_ids, state, _  = result[0]       
    
    # to reduce importance of high prob ones
    # if the same sequence is repeating we'll avoid it explicitly            
    n_probs = np.array([p for _,_,p in result[:10]])
    p_j = np.random.choice(list(range(n_probs.size)), p=n_probs/n_probs.sum())                    
    best_beam_ids, state, _ = result[p_j]
    x = np.array([[best_beam_ids[-1]]])
            
    text.extend([tokenizer.index_word[w] for w in best_beam_ids])    

print('\n')
print('='*60)
print("Final text: ")
print(''.join(text))

Making 54 predictions from input


Final text: 
chapter i. down the rabbit-hole alice was beginning to get very tired of sitting by her sister on the bank , and there was no reason that her father had brought him the story girl 's face .
 `` i 'm going to bed , '' said the prince , `` and you can not be able to do it . ''
 `` i 'm sure i shall have to go to bed , '' he answered , with a smile .
 `` i 'm so happy , '' she said .
 `` i do n't know how to bring you into the world , and i 'll be sure that you would have thought that it would have been a long time .
 there was no time to be able to do it , and it would have been a little thing . ''
 `` i do n't know , '' she said .
 `` i 'm so glad you come back . ''
 `` i do n't know what to do with you , '' said the princess , who would not have been able to do that .
 `` well , i do n't know , '' said anne .
 `` i 'm going to believe that i have n't anything more if i could , and i 'm glad you have a good deal .
 that is what it is . ''


In [ ]:

def beam_search(model, input_, state, beam_depth=5, beam_width=3):
    
    def recursive_fn(input_, state, sequence, log_prob, i):
        output, state = beam_one_step(model, input_, state)
        top_probs, top_ids = tf.nn.top_k(output, k=beam_width)
        top_probs, top_ids = top_probs.numpy().ravel(), top_ids.numpy().ravel()
        #print(top_probs, top_ids)
        while i < beam_depth:
            i += 1
            for p, wid in zip(top_probs, top_ids):
                log_prob += np.log(p)
                
                if log_prob > np.log(1e-6):                
                    sequence.append(wid)
                    sequence, prob, state = recursive_fn(np.array([[wid]]), state, sequence, log_prob, i)
                    #print(sequence)
                    if len(sequence)==beam_depth:
                        results.append((list(sequence), state, np.exp(log_prob)))

                    sequence.pop()

                log_prob -= np.log(p)
                
        return sequence, log_prob, state
    
    results = []
    sequence = []
    log_prob = 0.0
    recursive_fn(input_, state, sequence, log_prob, 0)    
    #print(results)
    results = sorted(results, key=lambda x: x[2], reverse=True)
    
    return results